In [1]:
import pandas as pd
import numpy as np
import os
import pickle

from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import supertrend, volume_level

c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest


In [2]:
klines_file = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m15/Completo_XRPUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
#klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m01/Lateral_BTCUSDT_0m01_2023-06-20_2023-10-09.DataFrame'
#klines_file = './backtest/klines/0m05/Lateral_BTCUSDT_0m05_2023-06-20_2023-10-09.DataFrame'
#klines_file = './backtest/klines/2d01/Lateral_BTCUSDT_2d01_2023-06-20_2023-10-09.DataFrame'
klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/1h01/Lateral_BTCUSDT_1h01_2023-06-20_2023-10-09.DataFrame'
#klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'

#klines_file = './backtest/klines/1h04/Bajista_BTCUSDT_1h04_2021-04-12_2021-07-12.DataFrame'
klines_file = './backtest/klines/1h01/Full_BTCUSDT_1h01_2019-01-01_2024-06-20.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Datos de precios históricos (por ejemplo, datos de cierre)
data = np.array(df['close']).reshape(-1, 1)
n = len(data)

# Preparación de datos para LSTM
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 5
X, y = prepare_data(data, n_steps)

# Definir modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Entrenamiento del modelo
model.fit(X, y, epochs=200, verbose=0)



In [6]:
from binance.client import Client
client = Client()

df = client.get_historical_klines(symbol='BTCUSDT', interval='1h', limit=500)
df = pd.DataFrame(df)
df = df.iloc[:, :6]
df.columns = ["datetime", "open", "high", "low", "close", "volume"]
df['open'] = df['open'].astype('float')
df['high'] = df['high'].astype('float')
df['low'] = df['low'].astype('float')
df['close'] = df['close'].astype('float')
df['volume'] = df['volume'].astype('float')
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') 
#df=df.set_index('datetime')



# Predicción
X_test = df[['close']] #df[['SMA']]
y_pred = model.predict(X_test)
df['predict'] = y_pred


16/16 [==============================] - 0s 865us/step


In [8]:

indicators = [
     #{'col': 'SMA','name': 'SMA','color': 'white','row': 1, 'mode':'lines',},
     #{'col': 'predict+','name': 'Pred +','color': 'green','row': 1, 'mode':'markers',},
     #{'col': 'predict-','name': 'Pred -','color': 'red','row': 1, 'mode':'markers',},
     {'col': 'predict','name': 'Predict','color': 'yellow','row': 1, 'mode':'lines',},
        ]

indicators_out = [
     #{'col': 'st_trend','name': 'ST','color': 'white','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
        #{'df':data,'col':'buy','name': 'BUY','color': 'green','symbol': 'circle-dot' }, 
        #{'df':data,'col':'sell','name': 'SELL','color': 'red','symbol': 'circle-dot' }, 
     ]
if 'volume' in df.columns:
        df.drop(columns=['volume'],inplace=True)

fig = fn.ohlc_chart(df,indicators=indicators,indicators_out=indicators_out,events=events) #
fig.show()